In [50]:
import pandas as pd
import numpy as np

In [51]:
# tuning
tuning=pd.read_csv("result/madelon/tuning_old.txt",sep=" ")

In [52]:
cumu_time=tuning["time"]

In [53]:
excute_time=[ cumu_time[i] if i==0 else cumu_time[i]-cumu_time[i-1] for i in range(len(cumu_time)) ]

In [54]:
tuning["excute_time"]=excute_time

In [55]:
tuning.head()

,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
0,GBDTBTt,0.0,0.1,0.0,50,20,0.302313,14,14
1,GBDTBTt,0.0,0.1,0.0,100,20,0.298211,19,5
2,GBDTBTt,0.0,0.1,0.0,200,20,0.292296,27,8
3,GBDTBTt,0.0,0.1,0.0,300,20,0.300259,43,16
4,GBDTBTt,0.0,0.1,0.0,500,20,0.304771,78,35


In [56]:
tuning.sort_values(by="RMSE",inplace=True)

In [57]:
BTt=tuning[tuning["name"]=="GBDTBTt"]

In [58]:
GBFS=tuning[tuning["name"]=="GBFS_model"]
GBFS_adapt=tuning[tuning["name"]=="GBFS_adapt_model"]

In [59]:
GBFS["excute_time"].mean()

85.58309037900875

In [60]:
GBFS_adapt["excute_time"].mean()

90.77597402597402

In [61]:
BTt["excute_time"].mean()

8.76530612244898

In [62]:
# tuning plots
GBFS.head()


,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
398,GBFS_model,4.0,0.10,0.000,1000,13,0.275439,63429,653
393,GBFS_model,4.0,0.10,0.000,100,11,0.283328,61727,31
395,GBFS_model,4.0,0.10,0.000,300,12,0.286277,61946,138
933,GBFS_model,4.0,0.05,0.005,200,12,0.287545,116533,43
403,GBFS_model,4.0,0.05,0.000,500,11,0.289643,63849,210


In [63]:
GBFS_adapt.head()

,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
47,GBFS_adapt_model,0.001,0.02,0.00,700,500,0.268365,4019,191
1922,GBFS_adapt_model,0.800,0.02,0.02,500,131,0.348938,198213,106
117,GBFS_adapt_model,0.008,0.10,0.00,700,500,0.353504,15923,552
2418,GBFS_adapt_model,0.400,0.05,0.05,300,88,0.354436,235065,57
1930,GBFS_adapt_model,0.800,0.01,0.02,700,222,0.354846,198961,145


In [64]:
BTt.head()

,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
1070,GBDTBTt,0.0,0.10,0.010,1000,20,0.281281,128043,15
1089,GBDTBTt,0.0,0.01,0.010,500,20,0.281611,128173,10
12,GBDTBTt,0.0,0.05,0.000,700,20,0.281752,274,39
559,GBDTBTt,0.0,0.01,0.005,1000,20,0.281926,82158,17
25,GBDTBTt,0.0,0.01,0.000,500,20,0.283002,495,17


In [65]:
tuning_new=pd.read_csv("result/madelon/tuning_new.txt",sep=" ")
cumu_time=tuning_new["time"]
excute_time=[ cumu_time[i] if i==0 else cumu_time[i]-cumu_time[i-1] for i in range(len(cumu_time)) ]
tuning_new["excute_time"]=excute_time

In [66]:
GBFS_adapt_new=tuning_new[tuning_new["name"]=="GBFS_adapt_model"]

In [67]:
GBFS_adapt_new["alpha"].unique()

array([0.   , 0.001, 0.002, 0.004, 0.008])

In [68]:
# BTt=BTt[BTt["alpha"]!=0.2]
# GBFS=GBFS[GBFS["alpha"]!=0.2]
# GBFS_adapt=GBFS_adapt[GBFS_adapt["alpha"]!=0.2]
# GBFS_adapt_new=GBFS_adapt_new[GBFS_adapt_new["alpha"]!=0.2]

In [69]:
sorted(GBFS["alpha"].unique())

[0.0, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2]

In [70]:
BTt.sort_values("RMSE").head()

,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
1070,GBDTBTt,0.0,0.10,0.010,1000,20,0.281281,128043,15
1089,GBDTBTt,0.0,0.01,0.010,500,20,0.281611,128173,10
12,GBDTBTt,0.0,0.05,0.000,700,20,0.281752,274,39
559,GBDTBTt,0.0,0.01,0.005,1000,20,0.281926,82158,17
25,GBDTBTt,0.0,0.01,0.000,500,20,0.283002,495,17


In [71]:
GBFS[GBFS["number_of_features"]<=25].sort_values("RMSE").head(20)

,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
398,GBFS_model,4.0,0.10,0.000,1000,13,0.275439,63429,653
393,GBFS_model,4.0,0.10,0.000,100,11,0.283328,61727,31
395,GBFS_model,4.0,0.10,0.000,300,12,0.286277,61946,138
933,GBFS_model,4.0,0.05,0.005,200,12,0.287545,116533,43
403,GBFS_model,4.0,0.05,0.000,500,11,0.289643,63849,210
436,GBFS_model,8.0,0.02,0.000,200,7,0.289794,69603,56
929,GBFS_model,4.0,0.10,0.005,700,11,0.291227,116248,150
446,GBFS_model,8.0,0.01,0.000,700,8,0.291490,70959,202
1466,GBFS_model,4.0,0.05,0.010,300,15,0.291737,160688,64
932,GBFS_model,4.0,0.05,0.005,100,11,0.292760,116490,21


In [72]:
GBFS_adapt[GBFS_adapt["number_of_features"]<=25].sort_values("RMSE").head(20)

,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
3417,GBFS_adapt_model,0.100,0.10,0.20,100,25,0.377049,297511,14
2447,GBFS_adapt_model,0.800,0.05,0.05,500,20,0.381962,237211,93
3416,GBFS_adapt_model,0.100,0.10,0.20,50,14,0.392574,297497,7
3452,GBFS_adapt_model,0.200,0.05,0.20,100,14,0.400393,299505,16
3388,GBFS_adapt_model,0.064,0.10,0.20,50,23,0.404678,295900,8
3461,GBFS_adapt_model,0.200,0.02,0.20,300,25,0.405162,299970,39
3460,GBFS_adapt_model,0.200,0.02,0.20,200,25,0.413214,299931,30
3423,GBFS_adapt_model,0.100,0.05,0.20,50,16,0.419135,297904,8
3395,GBFS_adapt_model,0.064,0.05,0.20,50,21,0.419972,296305,8
3439,GBFS_adapt_model,0.100,0.01,0.20,200,25,0.422027,298733,25


In [73]:
GBFS_adapt_new[GBFS_adapt_new["number_of_features"]<=25].sort_values("RMSE").head(20)

,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
783,GBFS_adapt_model,0.8,0.02,0.002,300,24,0.374145,60149,38
1047,GBFS_adapt_model,0.8,0.02,0.004,300,23,0.458266,71851,37
1317,GBFS_adapt_model,0.8,0.01,0.008,300,24,0.467183,82546,33
776,GBFS_adapt_model,0.8,0.05,0.002,200,17,0.467487,59835,25
1313,GBFS_adapt_model,0.8,0.02,0.008,1000,20,0.470076,82475,106
1048,GBFS_adapt_model,0.8,0.02,0.004,500,22,0.470089,71913,62
1319,GBFS_adapt_model,0.8,0.01,0.008,1000,23,0.478280,82701,105
246,GBFS_adapt_model,0.8,0.05,0.000,50,23,0.478736,24866,10
770,GBFS_adapt_model,0.8,0.10,0.002,200,16,0.483874,59544,30
1302,GBFS_adapt_model,0.8,0.05,0.008,50,12,0.485506,82031,6


In [25]:
GBFS_adapt_new.sort_values("RMSE")

,name,mu,shrinkage,alpha,iteration,number_of_features,RMSE,time,excute_time
232,GBFS_adapt_model,0.400,0.02,0.000,500,193,0.358700,23276,89
487,GBFS_adapt_model,0.400,0.05,0.001,100,137,0.360852,43540,15
230,GBFS_adapt_model,0.400,0.02,0.000,200,191,0.361267,23141,34
486,GBFS_adapt_model,0.400,0.05,0.001,50,135,0.363514,43525,9
745,GBFS_adapt_model,0.400,0.10,0.002,100,96,0.363982,58243,16
226,GBFS_adapt_model,0.400,0.05,0.000,500,142,0.364575,22589,132
237,GBFS_adapt_model,0.400,0.01,0.000,300,231,0.364784,23571,49
1023,GBFS_adapt_model,0.400,0.02,0.004,300,164,0.364975,70825,37
1031,GBFS_adapt_model,0.400,0.01,0.004,1000,209,0.365899,71270,119
718,GBFS_adapt_model,0.128,0.01,0.002,500,495,0.365951,56800,62


In [ ]:
GBFS_adapt_new["alpha"].unique()

In [ ]:
## tuning plots


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# for BTt

alphas=sorted(BTt["alpha"].unique())
shrinks=BTt["shrinkage"].unique()
iterations=BTt["iteration"].unique()
colours=['g','b','r','darkorange']
marks=['o','.','^']
plt.figure()
## put in the same plot
# lw = 2
# for i,sh in enumerate(shrinks):
#     for j,num in enumerate(iterations):
#         select=[ BTt["shrinkage"].iloc[i]==sh and BTt["iteration"].iloc[i]==num for i in range(len(BTt))]
#         ys=BTt[select].sort_values("alpha")["RMSE"]
#         plt.plot(alphas, ys, color=colours[i],
#          lw=lw, label=f"sh={sh}, itern={num}",marker=marks[j])
# plt.xlabel('alphas')
# plt.ylabel('RMSE')
# plt.title('Tuning plot for GBDTBTt')
# plt.legend(loc="lower right")
# plt.show()

# put in the different plot 1
lw = 2
for j,num in enumerate(iterations):
    for i,sh in enumerate(shrinks):
        select=[ BTt["shrinkage"].iloc[i]==sh and BTt["iteration"].iloc[i]==num for i in range(len(BTt))]
        ys=BTt[select].sort_values("alpha")["RMSE"]
        plt.plot(alphas, ys, color=colours[i],
         lw=lw, label=f"sh={sh}",marker='o')
    plt.xlabel('alphas')
    plt.ylabel('RMSE')
    plt.title(f'Tuning plot for GBDTBTt with itern={num}')
    plt.legend(loc="lower right")
    plt.show()


# put in the different plot 2
lw = 2
for i,sh in enumerate(shrinks):
    for j,num in enumerate(iterations):
        select=[ BTt["shrinkage"].iloc[i]==sh and BTt["iteration"].iloc[i]==num for i in range(len(BTt))]
        ys=BTt[select].sort_values("alpha")["RMSE"]
        plt.plot(alphas, ys, color='b',
         lw=lw, label=f"itern={num}",marker=marks[j])
    plt.xlabel('alphas')
    plt.ylabel('RMSE')
    plt.title(f'Tuning plot for GBDTBTt with shrinkage={sh}')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# for GBFS
alphas=sorted(GBFS["alpha"].unique())
shrinks=GBFS["shrinkage"].unique()
iterations=GBFS["iteration"].unique()
mus=sorted(GBFS["mu"].unique())
colours=['g','b','r','darkorange']
marks=['o','s','^']
plt.figure()
## put in the same plot
# lw = 2
# for i,sh in enumerate(shrinks):
#     for j,num in enumerate(iterations):
#         select=[ BTt["shrinkage"].iloc[i]==sh and BTt["iteration"].iloc[i]==num for i in range(len(BTt))]
#         ys=BTt[select].sort_values("alpha")["RMSE"]
#         plt.plot(alphas, ys, color=colours[i],
#          lw=lw, label=f"sh={sh}, itern={num}",marker=marks[j])
# plt.xlabel('alphas')
# plt.ylabel('RMSE')
# plt.title('Tuning plot for GBDTBTt')
# plt.legend(loc="lower right")
# plt.show()



# ## for fixed alpha
# for alpha in alphas:
#     GBFS_alpha=GBFS[GBFS["alpha"]==alpha]
#     # put in the different plot 1
#     lw = 2
#     for j,num in enumerate(iterations):
#         for i,sh in enumerate(shrinks):
#             select=[ GBFS_alpha["shrinkage"].iloc[i]==sh and GBFS_alpha["iteration"].iloc[i]==num for i in range(len(GBFS_alpha))]
#             ys=GBFS_alpha[select].sort_values("mu")["RMSE"]
#             plt.plot(mus, ys, color=colours[i],
#              lw=lw, label=f"sh={sh}",marker='o')
#         plt.xlabel('mu')
#         plt.ylabel('RMSE')
#         plt.title(f'Tuning plot for GBFS with itern={num} and alpha={alpha}')
#         plt.legend(loc="lower right")
#         plt.show()


#     # put in the different plot 2
#     lw = 2
#     for i,sh in enumerate(shrinks):
#         for j,num in enumerate(iterations):
#             select=[ GBFS_alpha["shrinkage"].iloc[i]==sh and GBFS_alpha["iteration"].iloc[i]==num for i in range(len(GBFS_alpha))]
#             ys=GBFS_alpha[select].sort_values("mu")["RMSE"]
#             plt.plot(mus, ys, color='b',
#              lw=lw, label=f"itern={num}",marker=marks[j])
#         plt.xlabel('mu')
#         plt.ylabel('RMSE')
#         plt.title(f'Tuning plot for GBFS with shrinkage={sh} and alpha={alpha}')
#         plt.legend(loc="lower right")
#         plt.show()

# fix mu=16
mu=16
GBFS_mu=GBFS[GBFS["mu"]==mu]
# put in the different plot 1
lw = 2
for j,num in enumerate(iterations):
    for i,sh in enumerate(shrinks):
        select=[ GBFS_mu["shrinkage"].iloc[i]==sh and GBFS_mu["iteration"].iloc[i]==num for i in range(len(GBFS_mu))]
        ys=GBFS_mu[select].sort_values("alpha")["RMSE"]
        plt.plot(alphas, ys, color=colours[i],
         lw=lw, label=f"sh={sh}",marker='o')
    plt.xlabel('alphas')
    plt.ylabel('RMSE')
    plt.title(f'Tuning plot for GBFS with itern={num} and mu={mu}')
    plt.legend(loc="lower right")
    plt.show()


# put in the different plot 2
lw = 2
for i,sh in enumerate(shrinks):
    for j,num in enumerate(iterations):
        select=[ GBFS_mu["shrinkage"].iloc[i]==sh and GBFS_mu["iteration"].iloc[i]==num for i in range(len(GBFS_mu))]
        ys=GBFS_mu[select].sort_values("alpha")["RMSE"]
        plt.plot(alphas, ys, color='b',
         lw=lw, label=f"itern={num}",marker=marks[j])
    plt.xlabel('alphas')
    plt.ylabel('RMSE')
    plt.title(f'Tuning plot for GBFS with shrinkage={sh} and mu={mu}')
    plt.legend(loc="lower right")
    plt.show()





In [ ]:
# for GBFS_adapt
alphas=sorted(GBFS_adapt["alpha"].unique())
shrinks=GBFS_adapt["shrinkage"].unique()
iterations=GBFS_adapt["iteration"].unique()
mus=sorted(GBFS_adapt["mu"].unique())
colours=['g','b','r','darkorange']
marks=['o','s','^']
plt.figure()
## put in the same plot
# lw = 2
# for i,sh in enumerate(shrinks):
#     for j,num in enumerate(iterations):
#         select=[ BTt["shrinkage"].iloc[i]==sh and BTt["iteration"].iloc[i]==num for i in range(len(BTt))]
#         ys=BTt[select].sort_values("alpha")["RMSE"]
#         plt.plot(alphas, ys, color=colours[i],
#          lw=lw, label=f"sh={sh}, itern={num}",marker=marks[j])
# plt.xlabel('alphas')
# plt.ylabel('RMSE')
# plt.title('Tuning plot for GBDTBTt')
# plt.legend(loc="lower right")
# plt.show()



## for fixed alpha
# for alpha in alphas:
#     GBFS_adapt_alpha=GBFS_adapt[GBFS_adapt["alpha"]==alpha]
#     # put in the different plot 1
#     lw = 2
#     for j,num in enumerate(iterations):
#         for i,sh in enumerate(shrinks):
#             select=[ GBFS_adapt_alpha["shrinkage"].iloc[i]==sh and GBFS_adapt_alpha["iteration"].iloc[i]==num for i in range(len(GBFS_adapt_alpha))]
#             ys=GBFS_adapt_alpha[select].sort_values("mu")["RMSE"]
#             plt.plot(mus, ys, color=colours[i],
#              lw=lw, label=f"sh={sh}",marker='o')
#         plt.xlabel('mu')
#         plt.ylabel('RMSE')
#         plt.title(f'Tuning plot for GBFS_adapt with itern={num} and alpha={alpha}')
#         plt.legend(loc="lower right")
#         plt.show()


#     # put in the different plot 2
#     lw = 2
#     for i,sh in enumerate(shrinks):
#         for j,num in enumerate(iterations):
#             select=[ GBFS_adapt_alpha["shrinkage"].iloc[i]==sh and GBFS_adapt_alpha["iteration"].iloc[i]==num for i in range(len(GBFS_adapt_alpha))]
#             ys=GBFS_adapt_alpha[select].sort_values("mu")["RMSE"]
#             plt.plot(mus, ys, color='b',
#              lw=lw, label=f"itern={num}",marker=marks[j])
#         plt.xlabel('mu')
#         plt.ylabel('RMSE')
#         plt.title(f'Tuning plot for GBFS_adapt with shrinkage={sh} and alpha={alpha}')
#         plt.legend(loc="lower right")
#         plt.show()

# # fix mu=0.16
mu=0.16
GBFS_adapt_mu=GBFS_adapt[GBFS_adapt["mu"]==mu]
# put in the different plot 1
lw = 2
for j,num in enumerate(iterations):
    for i,sh in enumerate(shrinks):
        select=[ GBFS_adapt_mu["shrinkage"].iloc[i]==sh and GBFS_adapt_mu["iteration"].iloc[i]==num for i in range(len(GBFS_adapt_mu))]
        ys=GBFS_adapt_mu[select].sort_values("alpha")["RMSE"]
        plt.plot(alphas, ys, color=colours[i],
         lw=lw, label=f"sh={sh}",marker='o')
    plt.xlabel('alphas')
    plt.ylabel('RMSE')
    plt.title(f'Tuning plot for GBFS_adapt with itern={num} and mu={mu}')
    plt.legend(loc="lower right")
    plt.show()


# put in the different plot 2
lw = 2
for i,sh in enumerate(shrinks):
    for j,num in enumerate(iterations):
        select=[ GBFS_adapt_mu["shrinkage"].iloc[i]==sh and GBFS_adapt_mu["iteration"].iloc[i]==num for i in range(len(GBFS_adapt_mu))]
        ys=GBFS_adapt_mu[select].sort_values("alpha")["RMSE"]
        plt.plot(alphas, ys, color='b',
         lw=lw, label=f"itern={num}",marker=marks[j])
    plt.xlabel('alphas')
    plt.ylabel('RMSE')
    plt.title(f'Tuning plot for GBFS_adapt with shrinkage={sh} and mu={mu}')
    plt.legend(loc="lower right")
    plt.show()



In [ ]:
## analysis tuned result for gold_in_qa_split
from os import listdir
from sklearn.metrics import roc_auc_score


In [ ]:
def ranking_precision_score(y_true, y_score, k=10):
    """Precision at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array-like, shape = [n_samples]
        Predicted scores.
    k : int
        Rank.
    Returns
    -------
    precision @k : float
    """
    unique_y = np.unique(y_true)

    if len(unique_y) == 2:
        pos_label = unique_y[1]
        n_pos = np.sum(y_true == pos_label)

        order = np.argsort(y_score)[::-1]
        y_true = np.take(y_true, order[:k])
        n_relevant = np.sum(y_true == pos_label)

        # Divide by min(n_pos, k) such that the best achievable score is always 1.0.
        return float(n_relevant) / min(n_pos, k)
    else:
        return -1
def rank_precision(data_frame,k=4): # passed to groupby object 
    y_true=list(data_frame[1])
    y_score=list(data_frame[2])
    return ranking_precision_score(y_true, y_score, k=k)

In [ ]:
class Model:
    def __init__(self,name=None,alpha=0,penalty=0,topk=-1,attrN=-1,shrink=0.1,
                 itern=100,preds=None,rmse=-1):
        self.name=name  # name can be GBDT or GBDTBT (GBDT with top ranking feature from BT)
        self.pen=penalty
        self.topk=topk
        self.preds=preds
        self.attrN=attrN
        self.alpha=alpha
        self.shrink=shrink
        self.itern=itern
        self.rmse=rmse
        
    def get_AUC(self):
        return roc_auc_score(trueY,self.preds) if self.preds is not None else None
    
    #add some other metrics later
    
    def get_ranking_precision(self,k=2): # ranking_precision_score averaged over queries
        combined=pd.concat([kw,trueY,self.preds],axis=1,ignore_index=True)
        grouped=combined.groupby(combined[0])
        agged=grouped.apply(rank_precision,k=k)
        agged=[num for num in agged if num>=0]
        return sum(agged)/(len(agged)) if len(agged)>0 else 0
        

In [ ]:
## result path

result_path='result/gold_in_qa_split/tuned/'
attr_total=96
# result_path='result/us/tuned/'
# attr_total=174

In [ ]:
filenames_result=listdir(result_path)
trueY=pd.read_csv(result_path+"trueY.txt",header=None)[0]
try:
    kw=pd.read_csv(result_path+"keyword.txt",header=None)[0]
except:
    print("do not have groups")
try:
    number_unused=len(pd.read_csv(result_path+"preprocess_unused.txt",header=None))
except:
    number_unused=0
attr_init_number=attr_total-number_unused

In [ ]:
filenames_result

In [ ]:
import collections
results_dict=collections.defaultdict(list)# name,mu,alpha,shrink,itern,attrn,rmse,auc,prec@k=2,prec@k=3,prec@k=4

In [ ]:
models=[]

In [ ]:
for fname in filenames_result:
    last_dot=fname.rfind(".")
    filename=fname[:last_dot]
    if "_preds" in filename:
        name=filename[:filename.index("_preds")]
        mu=float(filename[filename.index("_mu")+3:filename.index("_alpha")])
        alpha=float(filename[filename.index("_alpha")+6:filename.index("_shrink")])
        shrink=float(filename[filename.index("_shrink")+7:filename.index("_itern")])
        itern=int(filename[filename.index("_itern")+6:filename.index("_attrn")])
        attrn=int(filename[filename.index("_attrn")+6:filename.index("_rms")])
        rms=float(filename[filename.index("_rms")+4:])
        preds=pd.read_csv(result_path+fname,header=None)[0]
        models.append(Model(name=name,alpha=alpha,penalty=mu,attrN=attrn,shrink=shrink,
                 itern=itern,preds=preds,rmse=rms))
        
    

In [ ]:
for model in models:
    results_dict["name"].append(model.name)
    results_dict["mu"].append(model.pen)
    results_dict["shrink"].append(model.shrink)
    results_dict["itern"].append(model.itern)
    results_dict["attrn"].append(model.attrN)
    results_dict["alpha"].append(model.alpha)
    results_dict["rmse"].append(model.rmse)
    results_dict["AUC_ROC"].append(model.get_AUC())
    results_dict["prec@k=2"].append(model.get_ranking_precision(k=2))
    results_dict["prec@k=3"].append(model.get_ranking_precision(k=3))
    results_dict["prec@k=4"].append(model.get_ranking_precision(k=4))
    
    
    
    

In [ ]:
results_table=pd.DataFrame(results_dict)

In [ ]:
results_table.sort_values("name",inplace=True)

In [ ]:
results_table

In [ ]:
# split train data to train and validate

In [ ]:
# data_path="result/cpod/"
# data_name="cpod"
# data_path="result/baby/"
# data_name="baby"


In [ ]:
train=pd.read_csv(data_path+data_name+"_train.dta",sep="\t",header=None)

In [ ]:
queries_train=train[2].unique()

In [ ]:
validate_queries=set(np.random.choice(queries_train,replace=False,size=len(queries_train)//10))

In [ ]:
validate_data=train[[query in validate_queries for query in train[2]]]

In [ ]:
train_data=train[[query not in validate_queries for query in train[2]]]

In [ ]:
validate_data.to_csv(data_path+data_name+"_train_validate.dta",sep="\t",index=False,header=False)
train_data.to_csv(data_path+data_name+"_train_train.dta",sep="\t",index=False,header=False)

In [ ]:
# make entire query asin pair all in training or testing to do ranked precision/recall
# data_path="data/"
# data_name="gold_in_qa_split"
data_path="data/"
data_name="us_full"

In [ ]:
train=pd.read_csv(data_path+data_name+"_train.tsv",sep="\t")
test=pd.read_csv(data_path+data_name+"_test.tsv",sep="\t")
train_train=pd.read_csv(data_path+data_name+"_train_train.tsv",sep="\t")
train_validate=pd.read_csv(data_path+data_name+"_train_validate.tsv",sep="\t")

In [ ]:
len(train)

In [ ]:
# random split 10% train to form validation dataset according to query

In [ ]:
queries_train=train["keyword"].unique()

In [ ]:
queries_train_validate=set(np.random.choice(queries_train,replace=False,size=len(queries_train)//10))

In [ ]:
validate_data=train[[query in queries_train_validate for query in train["keyword"]]]

In [ ]:
train_data=train[[query not in queries_train_validate for query in train["keyword"]]]

In [ ]:
len(validate_data)

In [ ]:
len(train_data)

In [ ]:
validate_data.to_csv(data_path+data_name+"_train_validate.tsv",sep="\t",index=False)
train_data.to_csv(data_path+data_name+"_train_train.tsv",sep="\t",index=False)

In [ ]:
all_data=pd.concat([train,test],ignore_index=True)

In [ ]:
queries=all_data["keyword"].unique()

In [ ]:
len(all_data)/len(queries)

In [ ]:
np.random.seed(1)
test_queries=set(np.random.choice(queries,replace=False,size=(len(test)*len(queries))//len(all_data)))

In [ ]:
test_data=all_data[[query in test_queries for query in all_data["keyword"]]]

In [ ]:
train_data=all_data[[query not in test_queries for query in all_data["keyword"]]]

In [ ]:
len(test_data)/len(train_data),len(test)/len(train)

In [ ]:
test_data.to_csv(data_path+data_name+"_test.tsv",sep="\t",index=False)
train_data.to_csv(data_path+data_name+"_train.tsv",sep="\t",index=False)

In [ ]:
## deal with feature names

In [ ]:
train=pd.read_csv(data_path+data_name+"_train.tsv",sep="\t")
test=pd.read_csv(data_path+data_name+"_test.tsv",sep="\t")
train_train=pd.read_csv(data_path+data_name+"_train_train.tsv",sep="\t")
train_validate=pd.read_csv(data_path+data_name+"_train_validate.tsv",sep="\t")

In [ ]:
train.rename(columns={"search.alias":"search_alias"},inplace=True)
train_train.rename(columns={"search.alias":"search_alias"},inplace=True)
train_validate.rename(columns={"search.alias":"search_alias"},inplace=True)
test.rename(columns={"search.alias":"search_alias"},inplace=True)

In [ ]:
train_validate.to_csv(data_path+data_name+"_train_validate.tsv",sep="\t",index=False)
train_train.to_csv(data_path+data_name+"_train_train.tsv",sep="\t",index=False)
train.to_csv(data_path+data_name+"_train.tsv",sep="\t",index=False)
test.to_csv(data_path+data_name+"_test.tsv",sep="\t",index=False)

In [ ]:
## madelon dataset

In [ ]:
data_path="data/"
data_name="madelon"

In [ ]:
train=pd.read_csv(data_path+data_name+"_train.txt",sep=" ",header=None)
test=pd.read_csv(data_path+data_name+"_test.txt",sep=" ",header=None)

In [ ]:
train.head()

In [ ]:
train_data=train.iloc[:,1:].apply(lambda col: [int(r.split(":")[1]) for r in col])

In [ ]:
train_data[0]=[ int(x>0) for x in train[0]]

In [ ]:
test_data=test.iloc[:,1:].apply(lambda col: [int(r.split(":")[1]) for r in col])

In [ ]:
test_data[0]=[ int(x>0) for x in test[0]]

In [ ]:
test_data.rename(columns={0:"label"},inplace=True)

In [ ]:
train_data.rename(columns={0:"label"},inplace=True)

In [ ]:
test_data.to_csv(data_path+data_name+"_test.tsv",index=None,sep="\t")
train_data.to_csv(data_path+data_name+"_train.tsv",index=None,sep="\t")

In [ ]:
# random split 10% train to form validation dataset according to query
train=pd.read_csv(data_path+data_name+"_train.tsv",sep="\t")
from sklearn.model_selection import train_test_split
train, validate = train_test_split(train, test_size=0.1)
train.to_csv(data_path+data_name+"_train_train.tsv",index=None,sep="\t")
validate.to_csv(data_path+data_name+"_train_validate.tsv",index=None,sep="\t")

In [ ]:
from sklearn.model_selection import train_test_split
train, validate = train_test_split(train, test_size=0.1)

In [ ]:
train.to_csv(data_path+data_name+"_train_train.tsv",index=None,sep="\t")
validate.to_csv(data_path+data_name+"_train_validate.tsv",index=None,sep="\t")

In [ ]:
## gisette_scale data

In [ ]:
data_path="data/"
data_name="gisette_scale"

In [ ]:
train=pd.read_csv(data_path+data_name+"_train.txt",sep=" ",header=None)
test=pd.read_csv(data_path+data_name+"_test.txt",sep=" ",header=None)

In [ ]:
train_data=train.apply(lambda col: [ float(r.split(":")[1]) if type(r)==str and ":" in r else r for r in col])

In [ ]:
train_data[0]=[ int(x>0) for x in train_data[0]]

In [ ]:
test_data=test.apply(lambda col: [ float(r.split(":")[1]) if type(r)==str and ":" in r else r for r in col])

In [ ]:
test_data[0]=[ int(x>0) for x in test_data[0]]

In [ ]:
# test_data has more columns than train_data?

In [ ]:
test_data.rename(columns={0:"label"},inplace=True)
train_data.rename(columns={0:"label"},inplace=True)

In [ ]:
test_data=test_data.iloc[:,:4957]

In [ ]:
test_data.shape,train_data.shape

In [ ]:
test_data.to_csv(data_path+data_name+"_test.tsv",index=None,sep="\t")
train_data.to_csv(data_path+data_name+"_train.tsv",index=None,sep="\t")

In [ ]:
# random split 10% train to form validation dataset according to query
train=pd.read_csv(data_path+data_name+"_train.tsv",sep="\t")
from sklearn.model_selection import train_test_split
train, validate = train_test_split(train, test_size=0.1)
train.to_csv(data_path+data_name+"_train_train.tsv",index=None,sep="\t")
validate.to_csv(data_path+data_name+"_train_validate.tsv",index=None,sep="\t")

In [ ]:
import pandas as pd
import numpy as np
from os import listdir

In [ ]:
train="data/gisette_scale_train.tsv"
test="data/gisette_scale_test.tsv"
response_name="label"
folder="result/gisette_scale"

In [ ]:
train=pd.read_csv(train,sep="\t")
train_names=list(train.columns.values)
file_names=listdir(folder)

In [ ]:
allfeature=set(pd.read_csv(folder+"/allfeature.txt",header=None)[0])

In [ ]:
set(str(x) for x in allfeature)

In [ ]:
len(us_allfeature)

In [ ]:
pd.DataFrame(us_allfeature).to_csv("result/us/allfeature.txt",header=None,index=None)

In [ ]:
cpod_trueY=pd.read_csv("result/cpod/trueY.txt",header=None)[0]

In [ ]:
cpod_pred=pd.read_csv("result/cpod/preds.txt",header=None)[0]

In [ ]:
cpod_pred20=pd.read_csv("result/cpod/preds_top20.txt",header=None)[0]

In [ ]:
cpod_pred.head()

In [ ]:
len(cpod_pred20)

In [ ]:
rmse=(((cpod_pred-cpod_trueY)**2).sum()/len(cpod_pred20))**0.5

In [ ]:
rmse20=(((cpod_pred20-cpod_trueY)**2).sum()/len(cpod_pred20))**0.5

In [ ]:
rmse20

In [ ]:
rmse